# Modelos Lineares


In [1]:
# @title Lendo a planilha

import numpy as np
import pandas as pd

df = pd.read_csv("../export/dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


In [2]:
# Colocando dados em desvio

y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T

# ciclo de referência
ref_idx = 1
y_ref = np.tile(y[ref_idx, :], (len(y), 1))
u_ref = np.tile(u[ref_idx, :], (len(u), 1))

# --- Calcular desvios ---
y_desvio = y - y_ref
u_desvio = u - u_ref


# Gerando modelos lineares


# ARX


In [3]:
def gerar_ARX(y, u):
    # Construindo a matriz de regressão
    X = np.hstack([y[:-1, :], u[:-1, :]])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


theta = gerar_ARX(y_desvio, u_desvio)

print("Parâmetros estimados:\n", theta)

# Simulando saída estimada
N = len(y_desvio)

y_arx = np.zeros_like(y_desvio)
y_arx[0] = y_desvio[0]  # inicializa com o primeiro valor real

for k in range(1, N):
    phi = np.concatenate([y_arx[k - 1], u_desvio[k - 1]])
    y_arx[k] = phi @ theta

y_arx = y_arx + y_ref  # Convertendo o valor desvio em variavel de engenharia


Parâmetros estimados:
 [[ 5.42536034e-01 -1.35338261e-02  1.25441603e+00  1.52042372e+00
   1.18002620e-02]
 [ 1.16370280e+00  1.57216671e+00 -6.45586745e+00  6.66633888e+01
   4.41654574e+00]
 [ 1.32594951e-01 -1.90151050e-02  2.74741539e-01 -1.08335036e+00
  -8.21826456e-02]
 [-3.72588408e-02  1.21482056e-02 -1.58222034e-01  1.10318892e+00
   2.19848024e-02]
 [ 5.12535212e-01 -1.33565104e-01  7.38512716e+00 -2.80672442e+00
   7.13451502e-01]
 [-9.72461280e-03  1.12558123e-03  6.59944891e-03  7.30686277e-02
   4.65489342e-03]
 [-1.66161357e-02  1.98322328e-03  1.16483985e-01  2.03844294e-01
   1.24295149e-02]
 [-8.18849444e-04 -9.64536321e-04  3.86519133e-02 -2.28334752e-02
  -3.06257000e-03]
 [ 3.73763316e-02 -3.56268195e-03 -1.15336380e-01 -2.54369614e-01
  -1.75110518e-02]]


In [4]:
# Plotando resultados
from lib.plot import plot_comparison

plot_comparison(ciclos, y, y_arx, "ARX")


## Espaço de estados


In [5]:
A = theta[:5, :].T
B = theta[5:, :].T
C = np.eye(5)
D = np.zeros((5, 4))

x = y_desvio[0].copy()  # inicializa com a primeira saída
y_ss = np.zeros_like(y_desvio)
y_ss[0] = x

for k in range(1, N):
    x = A @ x + B @ u_desvio[k - 1]
    y_ss[k] = C @ x + D @ u_desvio[k - 1]

y_ss = y_ss + y_ref  # Convertendo o valor desvio em variavel de engenharia


In [6]:
# Plotando resultados
plot_comparison(ciclos, y, y_ss, "state-space")

print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D)


A: [[ 5.42536034e-01  1.16370280e+00  1.32594951e-01 -3.72588408e-02
   5.12535212e-01]
 [-1.35338261e-02  1.57216671e+00 -1.90151050e-02  1.21482056e-02
  -1.33565104e-01]
 [ 1.25441603e+00 -6.45586745e+00  2.74741539e-01 -1.58222034e-01
   7.38512716e+00]
 [ 1.52042372e+00  6.66633888e+01 -1.08335036e+00  1.10318892e+00
  -2.80672442e+00]
 [ 1.18002620e-02  4.41654574e+00 -8.21826456e-02  2.19848024e-02
   7.13451502e-01]]
B: [[-0.00972461 -0.01661614 -0.00081885  0.03737633]
 [ 0.00112558  0.00198322 -0.00096454 -0.00356268]
 [ 0.00659945  0.11648398  0.03865191 -0.11533638]
 [ 0.07306863  0.20384429 -0.02283348 -0.25436961]
 [ 0.00465489  0.01242951 -0.00306257 -0.01751105]]
C: [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
D: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
